In [245]:
import yfinance as yf
import pandas as pd
import datetime
import tensorflow as tf
import keras
import seaborn
import numpy as np

In [246]:
# Create fixed-window sequences for training and validation data
def create_sequences(X, window_size):
    seq_X = []
    seq_y = []
    for i in range(len(X) - window_size):
        seq_X.append(X[i:i+window_size])
        seq_y.append(X[i+window_size])
    return seq_X, seq_y

In [247]:
def preprocess(data):

    PF = np.polyfit(np.linspace(0,len(data),num=len(data)), np.log(data), 1)

    preprocessed = data / (np.exp(PF[0] * np.linspace(0,len(data),num=len(data)) + PF[1]))
    m = np.mean(preprocessed)
    s = np.std(preprocessed)
    preprocessed = (preprocessed - m)/s

    details = [m, s, PF]

    return preprocessed, details

In [248]:
df = pd.read_excel("/Users/lars/Documents/GitHub/NeuralNetworks_Assignment/M3C.xls")
df = df.iloc[:146,6:26]

df_train = df.iloc[:,:14]
df_test = df.iloc[:,14:]

window_size = 3

observations = []
PF = []

for index, row in df_train.iterrows():
    preprocessed, p = preprocess(np.array(row))
    PF.append(p)
   
    for i in range(len(preprocessed) - window_size):
        observations.append([preprocessed[i:i+window_size],preprocessed[i+window_size], p, i+window_size])

In [249]:
np.random.shuffle(observations)
train = observations[:int(np.floor(len(observations)*0.8))]
validation = observations[int(np.floor(len(observations)*0.8)):]


In [250]:
def build_model(x_train, y_train, x_validation, y_validation, window_size, options):
    # Build the FFNN model
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(window_size, 1))) 
    model.add(keras.layers.Dense(options[0][0], activation='relu'))

    if len(options[0]) > 1:
        if len(options[0]) > 2:
            for i in range(1,len(options[0])-1):
                model.add(keras.layers.Dense(options[0][i], activation=options[1]))
                
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer='adam', loss='mse', metrics=['mse'])

    # Train the model
    model.fit(x_train, y_train, epochs=options[3], batch_size=options[2], validation_data=(x_validation, y_validation), verbose = 0)

    # Make predictions
    predictions = model.predict(x_validation)

    # Evaluate the model
    loss, accuracy = model.evaluate(x_validation, y_validation)

    return model

In [251]:
x_train = []
y_train = []

for i in range(len(train)):
    x_train.append(train[i][0])
    y_train.append(train[i][1])

x_validation = []
y_validation = []

for i in range(len(validation)):
    x_validation.append(validation[i][0])
    y_validation.append(validation[i][1])

x_train = np.array(x_train).reshape(len(x_train),window_size)
y_train = np.array(y_train).reshape(len(y_train))
x_validation = np.array(x_validation).reshape(len(x_validation),window_size)
y_validation = np.array(y_validation).reshape(len(x_validation))

In [252]:
def get_data():

    df = pd.read_excel("/Users/lars/Documents/GitHub/NeuralNetworks_Assignment/M3C.xls")
    df = df.iloc[:146,6:26]

    df_train = df.iloc[:,:14]
    df_test = df.iloc[:,14:]

    window_size = 3

    observations = []
    PF = []

    for index, row in df_train.iterrows():
        preprocessed, p = preprocess(np.array(row))
        PF.append(p)
   
    for i in range(len(preprocessed) - window_size):
        observations.append([preprocessed[i:i+window_size],preprocessed[i+window_size], p, i+window_size])

    np.random.shuffle(observations)
    train = observations[:int(np.floor(len(observations)*0.8))]
    validation = observations[int(np.floor(len(observations)*0.8)):]

    x_train = []
    y_train = []

    for i in range(len(train)):
        x_train.append(train[i][0])
        y_train.append(train[i][1])

    x_validation = []
    y_validation = []

    for i in range(len(validation)):
        x_validation.append(validation[i][0])
        y_validation.append(validation[i][1])

    x_train = np.array(x_train).reshape(len(x_train),window_size)
    y_train = np.array(y_train).reshape(len(y_train))
    x_validation = np.array(x_validation).reshape(len(x_validation),window_size)
    y_validation = np.array(y_validation).reshape(len(x_validation))

    return x_train, y_train, x_validation, y_validation, window_size

In [253]:
def reprocess(y, details):
    mean = details[2][0]
    std = details[2][1]
    PF = details[2]
    time = details[3]
    
    return ((y * std) + mean) * np.exp(PF[0] * time + PF[1])


In [254]:
def smape(model, validation):
    smape = 0
    prediction = model.predict(x_validation)
    for i in range(len(validation)):
        observation = validation[i]
        pred = prediction[i]

        x_hat = reprocess(pred, observation)
        x = reprocess(observation[1], observation)

        smape += 2*np.abs(x-x_hat)/(x+x_hat)*100

    smape /= len(validation)

    return smape


In [255]:
# lays = [[500,500]]
# epochs = [15]
# batchSizes = [16]
# activationFunctions = ['sigmoid','relu']

# options = []

# for layer in lays:
#     for activation in activationFunctions:
#         for batchSize in batchSizes:
#             for epoch in epochs:
#                 options.append([layer, activation, batchSize, epoch, 0, 0])


# for i in range(len(options)):
#     smape_avg=[]
#     for j in range(10):
#         model = build_model(x_train, y_train, x_validation, y_validation, window_size, options[i])

#         print(options[i])
        
#         smape_avg.append(smape(model, validation))



#     options[i][4] = np.mean(smape_avg)
#     options[i][5] = np.std(smape_avg)


# op = pd.DataFrame(options)
# res = op.sort_values(4, ascending=False)
# print(res)

In [286]:
def get_smape(params):
    x_train, y_train, x_validation, y_validation, window_size = get_data()
    n_layers, l1, l2, l3, epochs, n_batch = params

    layers = [int(l1), int(l2), int(l3)]
    lays = []
    for i in range(int(n_layers)):
        lays.append(layers[i])
        
    acts = ['sigmoid','relu']

    options = [lays, 'sigmoid', int(n_batch), int(epochs), 0, 0]

    smape_avg=[]
    for j in range(10):
        model = build_model(x_train, y_train, x_validation, y_validation, window_size, options)
        smape_avg.append(smape(model, validation))

    return np.mean(smape_avg)

In [295]:
from geneticalgorithm import geneticalgorithm as ga

varbound = np.array([[1, 3], [2, 20], [2, 20], [2, 20], [1, 50], [1, 50]])
vartype = np.array([['int'], ['int'], ['int'], ['int'], ['int'], ['int']])

model = ga(function=get_smape, 
            dimension=6,
            variable_type_mixed=vartype, 
            variable_boundaries=varbound,
            function_timeout=240.0)

model.run()

11/11 [==============================] - 0s 553us/step


KeyboardInterrupt: 

In [294]:
print(model.output_dict)

{'variable': array([ 1., 14., 17.,  7., 42., 20.]), 'function': 13.960604667663574}


In [297]:
lays = [[2,2],[2,5],[5,2],[5,5],[2,2,2],[2,5,2],[5,5,5]]
epochs = [10,15,20,25,50]
batchSizes = [2,4,8,16,32,64]
activationFunctions = ['sigmoid','relu']

options = []

for layer in lays:
    for activation in activationFunctions:
        for batchSize in batchSizes:
            for epoch in epochs:
                options.append([layer, activation, batchSize, epoch, 0, 0])


for i in range(len(options)):
    smape_avg=[]
    for j in range(10):
        model = build_model(x_train, y_train, x_validation, y_validation, window_size, options[i])

        print(options[i])
        
        smape_avg.append(smape(model, validation))

    options[i][4] = np.mean(smape_avg)
    options[i][5] = np.std(smape_avg)


op = pd.DataFrame(options)
res = op.sort_values(4, ascending=False)
print(res)

11/11 [==============================] - 0s 599us/step - loss: 0.8029 - mse: 0.8029
[[2, 2], 'sigmoid', 2, 10, 0, 0]
11/11 [==============================] - 0s 606us/step - loss: 0.7779 - mse: 0.7779
[[2, 2], 'sigmoid', 2, 10, 0, 0]
11/11 [==============================] - 0s 591us/step - loss: 0.7813 - mse: 0.7813
[[2, 2], 'sigmoid', 2, 10, 0, 0]
11/11 [==============================] - 0s 628us/step - loss: 0.8514 - mse: 0.8514
[[2, 2], 'sigmoid', 2, 10, 0, 0]
11/11 [==============================] - 0s 665us/step - loss: 0.8521 - mse: 0.8521
[[2, 2], 'sigmoid', 2, 10, 0, 0]
11/11 [==============================] - 0s 604us/step - loss: 0.8569 - mse: 0.8569
[[2, 2], 'sigmoid', 2, 10, 0, 0]
11/11 [==============================] - 0s 601us/step - loss: 0.8750 - mse: 0.8750
[[2, 2], 'sigmoid', 2, 10, 0, 0]
11/11 [==============================] - 0s 590us/step - loss: 0.7888 - mse: 0.7888
[[2, 2], 'sigmoid', 2, 10, 0, 0]
11/11 [==============================] - 0s 591us/step - loss: 0

In [ ]:
##Testing
df = pd.read_excel("/Users/lars/Documents/GitHub/NeuralNetworks_Assignment/M3C.xls")
df = df.iloc[:146,6:26]

df_test = df.iloc[:,11:14]

window_size = 3
n_predictions = 6

observations = []
PF = []

for p in range(n_predictions):  

for index, row in df_test.iterrows():
    preprocessed, p = preprocess(np.array(row))
    PF.append(p)
   
for i in range(len(preprocessed) - window_size):
    observations.append([preprocessed[i:i+window_size],preprocessed[i+window_size], p, i+window_size])

In [ ]:
## Testing




lays = [500,500]
epochs = [15]
batchsize = [16]
activation = ['sigmoid']

options = [lays, activation, batchsize, epochs, 0, 0]

smape_avg=[]
for j in range(20):
    model = test_model(x_train, y_train, x_validation, y_validation, window_size, options)
    smape_avg.append(smape(model, validation))

    options[i][4] = np.mean(smape_avg)
    options[i][5] = np.std(smape_avg)

print(pd.DataFrame(options))